In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('finalDataset.csv')
df

,id,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year
0,7lmeHLHBe4nmXzuXc0HDjk,0,0.470,0.978,7,-5.399,1,0.0727,0.02610,0.000011,0.3560,0.503,117.906,210133,4,1999
1,1wsRitfRRtWyEapl0q22o8,1,0.599,0.957,11,-5.764,1,0.1880,0.01290,0.000071,0.1550,0.489,103.680,206200,4,1999
2,1hR0fIFK2qRG3f3RF70pb7,0,0.315,0.970,7,-5.424,1,0.4830,0.02340,0.000002,0.1220,0.370,149.749,298893,4,1999
3,2lbASgTSoDO7MTuLAXlTW0,1,0.440,0.967,11,-5.830,0,0.2370,0.16300,0.000004,0.1210,0.574,96.752,213640,4,1999
4,1MQTmpYOZ6fcMQc56Hdo7T,0,0.426,0.929,2,-6.729,1,0.0701,0.00162,0.105000,0.0789,0.539,127.059,205600,4,1999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99993,2V7MUfpnYxttoEpP8RORYr,0,0.565,0.762,5,-4.993,1,0.0441,0.28300,0.000003,0.0665,0.354,145.069,268640,4,2008
99994,54Y8hw7tJVhfiMl4fR6HM8,0,0.447,0.815,0,-3.539,1,0.0394,0.02340,0.000590,0.1150,0.417,122.126,194387,4,2008
99995,3onwqb1aDAhOEKk2z54MDz,0,0.610,0.687,1,-8.189,1,0.0321,0.03340,0.003800,0.5080,0.109,128.023,262933,4,2008
99996,15aq0vixWE6bnSV2OleGB0,0,0.601,0.835,8,-4.666,1,0.0374,0.03900,0.000032,0.1180,0.332,144.990,224800,4,2008


In [2]:
df.columns

Index(['id', 'explicit', 'danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'duration_ms', 'time_signature', 'year'],
      dtype='object')

K - Nearest Neighbours algorithm

In [3]:
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
import joblib

features = ['explicit', 'danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'duration_ms', 'time_signature', 'year']

# Extracting features and song IDs
X = df[features]
song_ids = df['id']

# Scaling the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Initialize the Nearest Neighbors model
k_neighbors = 15  # the number of neighbors 
model = NearestNeighbors(n_neighbors=k_neighbors, metric='euclidean')
model.fit(X_scaled)

# Save the trained model using joblib
model_filename = 'knn_model.joblib'
joblib.dump(model, model_filename)

['knn_model.joblib']

In [4]:
allFeaturedDataset = pd.read_csv('tracks_features.csv')

# function to recommend song based on a given song ID and output its name and artist name from the ID:
def recommendSong(songId):
    # Get the index of the song ID
    song_index = df.index[df['id'] == songId].tolist()  # Ensure it's a list

    if not song_index:  # Check if the song_id exists
        raise ValueError(f"Song ID '{songId}' not found in the dataset.")
    
    # Extract the first matching index (assuming IDs are unique)
    song_index = song_index[0]

    # Extract the features for the query song
    query_song_features = X_scaled[song_index].reshape(1, -1)

    # Find the k nearest neighbours of the queried song
    distances, indices = model.kneighbors(query_song_features)

    # Retrieve and return the recommended songs using the indices
    recommended_songs = df.iloc[indices[0]]['id'].tolist()
    return recommended_songs

In [5]:
input_song_id = '7lmeHLHBe4nmXzuXc0HDjk' 
recommended_songs = recommendSong(input_song_id)

# Display the recommended songs
print("Recommended Songs:")
print(recommended_songs)

Recommended Songs:
['7lmeHLHBe4nmXzuXc0HDjk', '1Udy7blrWwKMOlpQKlQS82', '5CfUrUz4mtLXG5sbtqs6MG', '5fFKME0Kpvs71nO1wWvDRT', '43mywkNjFsKXnSmEcb2dKK', '1ki35qkMVQfnyAaHmO2L4B', '7J6m1TwmgpJi02zhsjwzMK', '0n79NeB8blgNQmI3ZLeBXm', '1fvYAeEJZDmG4O45rL0jYc', '7ghsJcgJadxrP1y9NEzzNM', '7nb5q60nCrq82KL7Pys5u8', '6V72lIG20zIyyrWXbpGjba', '2QyFhSLwA4dkjDoXO7caeW', '6SKVR89W4mxNfboJRTZ8Ie', '4MsJcYxTfxhMX4joyVe6K9']
